                                                 Домашнє завдання N 3

                                            Лінійна регресія та її реалізація

Імпортуємо потрібні бібліотеки:

In [29]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler

Завантаження та вивід перших рядків датасету Housing.csv:

In [30]:
housing_data = pd.read_csv('Housing.csv')

housing_data.head()

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished


Знаходимо найкращі параметри w(омега) для датасету прогнозуючу ціну на будинок залежно
 від площі, кількості ванних кімнат та кількості спалень:

In [32]:
# Вибір ознак і цільової змінної
X = housing_data[['area', 'bedrooms', 'bathrooms']].values
y = housing_data['price'].values

# Нормалізація ознак
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Додавання колонки з одиницями до ознак для врахування вільного члена
X_scaled_with_bias = np.c_[np.ones(X_scaled.shape[0]), X_scaled]

# Відображення перших кількох рядків нормалізованих даних з одиницями
X_scaled_with_bias[:5]

array([[ 1.        ,  1.04672629,  1.40341936,  1.42181174],
       [ 1.        ,  1.75700953,  1.40341936,  5.40580863],
       [ 1.        ,  2.21823241,  0.04727831,  1.42181174],
       [ 1.        ,  1.08362412,  1.40341936,  1.42181174],
       [ 1.        ,  1.04672629,  1.40341936, -0.57018671]])

Створимо функцію для обчислення функції втрат у векторному вигляді:

In [ ]:
def cost(h, y):
  return np.mean((h-y)**2)/2

Створимо клас для лінійної регресії CustomLinearRegression:

In [33]:
class CustomLinearRegression:
  def __init__(self, lr: float = 0.01, thr=0.00001, n_epochs = 100):
    self.lr = lr
    self.thr = thr
    self.n_epochs = n_epochs
    self.w = np.random.rand(X_scaled_with_bias.shape[1])

  # Напишіть функцію гіпотези лінійної регресії у векторному вигляді
  def predict(self, X):
    return self.w@X.T
  
  # Реалізуйте один крок градієнтного спуску
  def update_weights(self, X, y):
    m = len(y)
    h = self.predict(X)
    self.w = self.w - self.lr / m * X.T @ (h-y)
  
  def fit(self, X, y):
    last_cost = 10000000
    for i in range(self.n_epochs):
      self.update_weights(X, y)
      new_cost = cost(self.predict(X), y)

      if i % (self.n_epochs//100) == 0:
        print(f"cost function = {last_cost}")

      if abs(last_cost - new_cost) < self.thr:
        break

      last_cost = new_cost

Ініціалізація на тренування CustomLinearRegression моделі:

In [35]:
custom_lr = CustomLinearRegression(lr=0.01, thr=0.00001, n_epochs=1000)
w_custom_lr = custom_lr.fit(X_scaled_with_bias, y)

cost function = 10000000
cost function = 10827793277504.66
cost function = 8978437435955.99
cost function = 7476706039536.387
cost function = 6256434303814.798
cost function = 5264256183789.982
cost function = 4457077772970.173
cost function = 3800061891335.545
cost function = 3265017465082.2124
cost function = 2829110140895.201
cost function = 2473828356488.5825
cost function = 2184152963482.0762
cost function = 1947889352973.0544
cost function = 1755129546781.9932
cost function = 1597818409726.133
cost function = 1469403412167.2605
cost function = 1364551537793.0051
cost function = 1278920229396.995
cost function = 1208971881827.6687
cost function = 1151823471415.9045
cost function = 1105124568296.2312
cost function = 1066958300661.25
cost function = 1035760897565.1573
cost function = 1010256283988.9911
cost function = 989402881488.5161
cost function = 972350313813.6798
cost function = 958404156284.5621
cost function = 946997221743.8922
cost function = 937666161519.5428
cost function

Знаходимо ці ж параметри за допомогою аналітичного рішення:

In [37]:
def analytical_solution(X, y):
    return np.linalg.pinv(X) @ y

Застосування аналітичного рішення для пошуку найкращих параметрів:

In [38]:
w_analytical = analytical_solution(X_scaled_with_bias, y)

In [39]:
w_custom_lr, w_analytical

(None,
 array([4766729.24770642,  821214.14349519,  299983.57107963,
         695808.52272537]))